### TASK
*   Convert the payroll report into a tabular form with one header row.
*   Send back the output table(s) created and any scripts/code/Excel workbooks

### MY APPROACH
1. Explored CSV file to get rough idea about the given data and its structure, and to think about how I convert the report into a tabular form with one header row.
2. Checked column data uniqueness to get detail information about the given data and to identify what data I could clean.
3. Removed unnecessary columns.
4. Splitted given CSV into smaller dataframes and put it to the easier structure.

*   First dictionary: {emp_id, emp_salary_details}
*   Second dictionary: {emp_id, emp_salary_totals}
*   Third dictionary: {company total and average, respective amounts}

  *Please note that dictionary values were in dataframe structure.*

5. Removed unnecessary rows.
6. Removed unnecessary columns.
7. Filled empty dates in emp_salary_details dataframe using ffill() function to make next step (reshaping dataframe) easier.
8. Reshaped dataframes using unstack() function to make dataframe with a one header.
9. Concatenated all dataframes to the one big dataframe.
10. Exported resulted dataframe into CSV.

### IDENTIFIED ISSUES
1. Missing values <br>
1 week (between 1983/08/22 and 1983/09/05) was missing for 009KENTG	Garry Kent salary details.
1 week (between 1982/07/19 and 1982/08/02) was missing for 021MOODL Luann Moody salary details
2. 1983/08/22 and 1983/09/05 salary details were identical for 009KENTG	Garry Kent salary details.
3. Suspicious description:
<br> ZZZFirst Aid -hourly retail award (don't us <br>
e)


### CODE SCRIPT WITH BRIEF DESCRIPTION

*Please note that you need to rename given data as payroll_raw_data.csv and upload it to the content folder in google colab to run the code.*

**THE FIRST 3 STEPS APPLIED AS FOLLOWS:**
1. Explored CSV file to get rough idea about the given data and its structure, and to think about how I convert the report into a tabular form with one header row.
2. Checked column data uniqueness to get detail information about the given data and to identify what data I could clean.
3. Removed unnecessary columns.






In [ ]:
import pandas as pd

In [ ]:
# import given raw data
raw_data = pd.read_csv('/content/payroll_raw_data.csv', header=None)
print('Raw data shape: ', raw_data.shape)
raw_data.head(10)

Raw data shape:  (30550, 12)


,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,NaN,NaN,NaN,NaN,Employee Previous Earnings,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,AMALGAMATED WIDGETS PTY LTD,NaN,NaN,NaN,NaN,NaN,NaN
2,Employee Previous Earnings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Page 1 of 553
3,Payroll Company,NaN,AMALGAMATED WIDGETS - AMALGAMATED WIDGETS PTY LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001POWED,Dominick Powers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,For Pay End Periods between 1/07/1982 and 25/0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,Number of,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Period End Date,Pay Frequency,NaN,Location,NaN,Description,NaN,NaN,NaN,NaN,Hours,Amount
8,1982/12/06,1W01 - Weekly,NaN,CARCOSA,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,Normal Hours,NaN,NaN,NaN,NaN,23,351.87


From above result, we can see that the given file has 30550 rows and 12 columns and it includes employees' weekly salary information. <br> In order to understand the data throughly, let's have a look uniqueness of data column by column.

In [ ]:
print(raw_data[raw_data.columns[0]].unique())

[nan 'Employee Previous Earnings' 'Payroll Company' '001POWED'
 'For Pay End Periods between 1/07/1982 and 25/06/1985' 'Period End Date'
 '1982/12/06' '1982/12/13' '1982/12/20' '1982/12/27' '1983/01/03'
 '1983/01/10' '1983/01/17' '1983/01/24'
 'ACME Payroll (Registered to AMALGAMATED CO. PTY. LTD.)' '1983/01/31'
 '1983/02/07' '1983/02/14' '1983/02/21' '1983/02/28' '1983/03/07'
 '1983/03/14' '1983/03/21' '1983/03/28' '1983/04/04' '1983/04/11'
 '1983/04/18' '1983/04/25' '1983/05/02' '1983/05/09' '1983/05/16'
 '1983/05/23' '1983/05/30' 'Employee Total' '002MORIC' '1985/04/16'
 '1985/04/23' '1985/04/30' '1985/05/07' '003DALTE' '1982/07/05'
 '1982/07/12' '1982/07/19' '1982/07/26' '1982/08/02' '1982/08/09'
 '1982/08/16' '1982/08/23' '1982/08/30' '1982/09/06' '1982/09/13'
 '1982/09/20' '1982/09/27' '1982/10/04' '1982/10/11' '1982/10/18'
 '1982/10/25' '1982/11/01' '1982/11/08' '1982/11/15' '1982/11/22'
 '1982/11/29' '1983/06/06' '1983/06/13' '1983/06/20' '1983/06/27'
 '1983/07/04' '1983/07/11'

In column 0, important information are **period end dates** and **employee ids**. Also, we could notice that there are employee total and company total and average data in the given file because those labels are in this column. In addition, we could judge this file includes 48 employees' 3 years salary data due to given date interval **between 1/07/1982 and 25/06/1985** and the last employee id **048LOWEP**.

In [ ]:
print(raw_data[raw_data.columns[1]].unique())

[nan 'Dominick Powers' 'Pay Frequency' '1W01 - Weekly' 'Crystal Morin'
 'Erma Dalton' 'Kristy Nash' 'Henrietta Randall' 'Avis Browning'
 'Abigail Gaines' 'Randell Calhoun' 'Garry Kent' 'Melanie Golden'
 'Jonathan Mcconnell' 'Merrill Gamble' 'Mauro Dixon' 'Sara Ellis'
 'Phil Rush' 'Beth Lucas' 'Nola Sims' 'Larry Delaney' 'Christy Mcintyre'
 'Milo Lyons' 'Luann Moody' 'Hans Torres' 'Velma Alford' 'Imogene Delgado'
 'Irving Pace' 'Georgina Ellison' 'Corinne Bryant' 'Stefan Miles'
 'Simon Ball' 'Rosanna Conway' 'Charley Montgomery' 'Cathy Beard'
 'Ryan Todd' 'Sammy Williamson' 'Eunice Rodriguez' 'Mercedes Dean'
 'Brandie Wall' 'Ethel Carney' 'Leonel Carson' 'Luz Mendez' 'Etta Kerr'
 'Claudio George' 'Anibal Lucas' 'Charlie Figueroa' 'Bertha West'
 'Leanne Mcgowan' 'Tamara Bailey' 'Patty Lowe']


In column 1, important information is **employee names**. Also, we could see payment frequency is weekly from the above information.

In [ ]:
print(raw_data[raw_data.columns[2]].unique())

[nan 'AMALGAMATED WIDGETS - AMALGAMATED WIDGETS PTY LTD']


In [ ]:
print(raw_data[raw_data.columns[3]].unique())

[nan 'Location' 'CARCOSA']


In [ ]:
print(raw_data[raw_data.columns[4]].unique())

[nan 'Number of' '1' '2' '5']


In column 2, we can see company name. <br>
In column 3, we can see that location is CARCOSA. <br>
In column 4, we could see number of something.

In [ ]:
print(raw_data[raw_data.columns[5]].unique())

['Employee Previous Earnings' 'AMALGAMATED WIDGETS PTY LTD' nan
 'Description' 'Normal Hours' 'Gross Taxable Total' 'Tax (Incl Adjust)'
 'Net Pay' 'Superannuation' 'Public Holiday - worked' 'Time/Half Hours'
 'Reimburse expenses' 'Uniforms - Reducing Balance' 'Sat Casual Ldg'
 'Other Leave - Public Holiday' 'Annual Leave' 'Sick Leave'
 'Other Leave - Birthday' 'Term AL Gross' 'Account Payment'
 '60% Sun Perm Ldg' 'Leave Loading 17.5%' 'Sunday Ldg'
 'Other Leave - Unpaid Leave- No accruals' 'Long Service Leave'
 'Term LL Gross' 'ETP - Taxable (Post June 83) Code : O' '   Gratuity'
 'Sat Perm Loading' 'Term LSL Gross' 'Sun Cas Ldg'
 'Other Leave - Jury Duty' 'Jury Duty Subsidy'
 'Reimburse expenses- Flu Vax' 'Other Leave - Bereavement'
 'Other Leave - Leave w/out pay- with accr' 'uals'
 'SUPER SUPER VOL CONT $' "ZZZFirst Aid -hourly retail award (don't us"
 'e)' 'First Aid- Retail award - wkly' 'First Aid - Retail Hourly Rate'
 'Re-imburse expense - Safety Boots'
 'Other Leave - Sick Lea

In column 5, we can see salary **pay description** basically, which is important to keep.

In [ ]:
print(raw_data[raw_data.columns[6]].unique())

[nan]


In [ ]:
print(raw_data[raw_data.columns[7]].unique())

[nan]


In [ ]:
print(raw_data[raw_data.columns[8]].unique())

[nan]


In [ ]:
print(raw_data[raw_data.columns[9]].unique())

[nan '2022/06/30 00:00:00.653']


We can say that column 6, column 7, column 8 and column 9 are unnecessary as 3 of them are empty and one of them has only 1 unique value.

In [ ]:
print(raw_data[raw_data.columns[10]].unique())

[nan 'Hours' '23' '0' '46' '37.5' '30' '15' '32' '5.5' '31' '37' '33.5'
 '35.5' '35.25' '34.5' '33.25' '41.5' '16.5' '8.5' '41.75' '25' '34'
 '14.5' '852' '23.5' '19.5' '4' '99.5' '40' '8' '10' '32.5' '7.5' '28'
 '12' '196.83' '2170' '29.5' '56.5' '16' '28.5' '29' '21' '25.5' '3' '4.5'
 '14' '7' '27.5' '20.5' '6.5' '27.25' '29.25' '26.5' '21.5' '28.75' '3.5'
 '17' '26' '5' '22.25' '6.25' '1.25' '2' '22.5' '6' '34.25' '3771.25'
 '63.25' '377' '65.5' '50' '693.75' '102.5' '307.5' '38' '76' '18' '20'
 '61' '36.25' '1.75' '37.25' '0.75' '25.75' '12.25' '38.5' '17.75' '20.25'
 '60.47' '190' '621' '58' '66' '342' '43' '40.5' '33' '9.25' '92.5' '12.5'
 '11' '11.5' '27' '17.5' '1' '18.75' '3.75' '24.5' '795' '104' '75'
 '109.5' '146.25' '35' '24' '15.5' '15.25' '6.94' '264.08' '720.5' '43.5'
 '117' '19' '9.5' '9' '29.75' '22' '8.25' '10.5' '36.5' '41' '39' '35.75'
 '45' '36.75' '2443.25' '72.5' '288' '244.5' '31.5' '32.25' '13.5' '13'
 '3466.25' '71.5' '267' '39.5' '138' '1.5' '32.75' '33.75' 

In [ ]:
print(raw_data[raw_data.columns[11]].unique())

[nan 'Page 1 of 553' 'Amount' ... '-0.1789104947097512'
 '636.7198341435585' '68.33625107234788']


Column 10 includes hours while column 11 includes amount.

**After observed uniqueness of data column by column, we could summarize as folows:**

**Columns to keep:**<br>
Column 0 - Pay end dates and employee ids <br>
Column 1 - Employee names <br>
Column 5 - Descriptions <br>
Column 10 - Hours <br>
Column 11 - Amount <br>

**Columns to remove:**<br>
Column 2 has only one unique value ([nan 'AMALGAMATED WIDGETS - AMALGAMATED WIDGETS PTY LTD']). <br>
Column 3 has only one unique value ([nan 'Location' 'CARCOSA']). <br>
Column 4 has 4 unique values ([nan 'Number of' '1' '2' '5']) but it is not quite relevant to the rest data.  <br>
Column 6 - Empty<br>
Column 7 - Empty<br>
Column 8 - Empty<br>
Column 9 has only one unique value ([nan '2022/06/30 00:00:00.653']) <br>

Although we remove some columns, we should be aware of some data in it for the future. So that, let's keep them here.<br>

Payroll company: **AMALGAMATED WIDGETS - AMALGAMATED WIDGETS PTY LTD** <br>
Location: **CARCOSA**


In [ ]:
#remove columns by indexing
raw_data = raw_data.drop(raw_data.columns[[2, 3, 4, 6, 7, 8, 9]], axis=1)

In [ ]:
print("Raw data shape after removed unnecessary columns: ", raw_data.shape)
raw_data.head(20)

Raw data shape after removed unnecessary columns:  (30550, 5)


,0,1,5,10,11
0,NaN,NaN,Employee Previous Earnings,NaN,NaN
1,NaN,NaN,AMALGAMATED WIDGETS PTY LTD,NaN,NaN
2,Employee Previous Earnings,NaN,NaN,NaN,Page 1 of 553
3,Payroll Company,NaN,NaN,NaN,NaN
4,001POWED,Dominick Powers,NaN,NaN,NaN
5,For Pay End Periods between 1/07/1982 and 25/0...,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,Period End Date,Pay Frequency,Description,Hours,Amount
8,1982/12/06,1W01 - Weekly,NaN,NaN,NaN
9,NaN,NaN,Normal Hours,23,351.87


We could see there are some unnecessary rows but let's keep those rows at this stage to avoid indices confusion for splitting a big dataframe. We will remove unnecessary rows in later stages.

**THE 4TH STEP APPLIED AS FOLLOWS:**
4. Splitted given CSV into smaller dataframes and put it to the easier structure.

*   First dictionary: {emp_id, emp_salary_details}
*   Second dictionary: {emp_id, emp_salary_totals}
*   Third dictionary: {company total and average, respective amounts}

  *Please note that dictionary values were in dataframe structure.*

Collected all employees ids from the unique output of column 0 to use them as keys in dictionary structure:


'001POWED' '002MORIC' '003DALTE' '004NASHK''005RANDH' '006BROWA' '007GAINA' '008CALHR' '009KENTG' '010GOLDM' '011MCCOJ' '012GAMBM' '013DIXOM' '014ELLIS' '015RUSHP' '016LUCAB' '017SIMSN' '018DELAL' '019MCINC' '020LYONM' '021MOODL' '022TORRH' '023ALFOV' '024DELGI' '025PACEI' '026ELLIG' '027BRYAC' '028MILES' '029BALLS' '030CONWR' '031MONTC' '032BEARC' '033TODDR' '034WILLS' '035RODRE' '036DEANM' '037WALLB' '038CARNE' '039CARSL' '040MENDL' '041KERRE' '042GEORC' '043LUCAA' '044FIGUC' '045WESTB' '046MCGOL' '047BAILT' '048LOWEP'

In [ ]:
# list of employee ids
list_emp_ids = ['001POWED', '002MORIC', '003DALTE', '004NASHK', '005RANDH', '006BROWA', '007GAINA', '008CALHR', '009KENTG', '010GOLDM', '011MCCOJ', '012GAMBM', '013DIXOM', '014ELLIS', '015RUSHP', '016LUCAB', '017SIMSN', '018DELAL', '019MCINC', '020LYONM', '021MOODL', '022TORRH', '023ALFOV', '024DELGI', '025PACEI', '026ELLIG', '027BRYAC', '028MILES', '029BALLS', '030CONWR', '031MONTC', '032BEARC', '033TODDR', '034WILLS', '035RODRE', '036DEANM', '037WALLB', '038CARNE', '039CARSL', '040MENDL', '041KERRE', '042GEORC', '043LUCAA', '044FIGUC', '045WESTB', '046MCGOL', '047BAILT', '048LOWEP']
# list to keep indices which we can use for splitting CSV by each employee.
list_indices = []
# dictionary to keep employee ids as keys and employee names as values.
dic_emp_names = {}

for id in list_emp_ids:
  index = raw_data[raw_data.iloc[:,0] == id].index[0]
  list_indices.append(index-4)
  dic_emp_names[id] = raw_data.loc[index, 1]

list_index_by_emp = list(zip(list_emp_ids, list_indices))

print("List of indices:\n", list_indices)
print("List of indices with respective employee ids:\n", list_index_by_emp)
print("Dictionary of employee names:\n", dic_emp_names.items())


List of indices:
 [0, 207, 247, 680, 2087, 2123, 2349, 2778, 3055, 3784, 4869, 6070, 7078, 8329, 9908, 10750, 11005, 11393, 11420, 11921, 12035, 13426, 13456, 14787, 16232, 16899, 16985, 17356, 17445, 17597, 17759, 18188, 18771, 19203, 20478, 20583, 20719, 20798, 22467, 22554, 22729, 24301, 24847, 25258, 26084, 27442, 28800, 30006]
List of indices with respective employee ids:
 [('001POWED', 0), ('002MORIC', 207), ('003DALTE', 247), ('004NASHK', 680), ('005RANDH', 2087), ('006BROWA', 2123), ('007GAINA', 2349), ('008CALHR', 2778), ('009KENTG', 3055), ('010GOLDM', 3784), ('011MCCOJ', 4869), ('012GAMBM', 6070), ('013DIXOM', 7078), ('014ELLIS', 8329), ('015RUSHP', 9908), ('016LUCAB', 10750), ('017SIMSN', 11005), ('018DELAL', 11393), ('019MCINC', 11420), ('020LYONM', 11921), ('021MOODL', 12035), ('022TORRH', 13426), ('023ALFOV', 13456), ('024DELGI', 14787), ('025PACEI', 16232), ('026ELLIG', 16899), ('027BRYAC', 16985), ('028MILES', 17356), ('029BALLS', 17445), ('030CONWR', 17597), ('031MONT

In [ ]:
# creating dictionary with employee ids as keys and salary details dataframe as values
start_index = 0
key_emp_id = ''
dic_emp_salary = {}

for emp_id, index in list_index_by_emp:
  if index == 0:
    key_emp_id = emp_id
    continue

  dic_emp_salary[key_emp_id] = raw_data.iloc[start_index:index,:]

  start_index = index
  key_emp_id = emp_id

dic_emp_salary[key_emp_id] = raw_data.iloc[start_index:,:]

print("The last employee ID: " + key_emp_id)

The last employee ID: 048LOWEP


In [ ]:
print("The 1st employee dataframe shape: ", dic_emp_salary['001POWED'].shape)
print("The 2nd employee dataframe shape: ", dic_emp_salary['002MORIC'].shape)
print("The 3rd employee dataframe shape: ", dic_emp_salary['003DALTE'].shape)
print("The 20th employee dataframe shape: ", dic_emp_salary['020LYONM'].shape)
print("The 21st employee dataframe shape: ", dic_emp_salary['021MOODL'].shape)
print("The 47th employee dataframe shape: ", dic_emp_salary['047BAILT'].shape)
print("The 48th employee dataframe shape: ", dic_emp_salary['048LOWEP'].shape)

The 1st employee dataframe shape:  (207, 5)
The 2nd employee dataframe shape:  (40, 5)
The 3rd employee dataframe shape:  (433, 5)
The 20th employee dataframe shape:  (114, 5)
The 21st employee dataframe shape:  (1391, 5)
The 47th employee dataframe shape:  (1206, 5)
The 48th employee dataframe shape:  (544, 5)


After quick look at raw data file, we can confirm the shape of dataframes for above employees are correct. However, the last employee's dataframe includes company total and company average data, so that we should take care of that in the next steps.

Let's have a look some employees dataframes below.

In [ ]:
dic_emp_salary['001POWED']

,0,1,5,10,11
0,NaN,NaN,Employee Previous Earnings,NaN,NaN
1,NaN,NaN,AMALGAMATED WIDGETS PTY LTD,NaN,NaN
2,Employee Previous Earnings,NaN,NaN,NaN,Page 1 of 553
3,Payroll Company,NaN,NaN,NaN,NaN
4,001POWED,Dominick Powers,NaN,NaN,NaN
...,...,...,...,...,...
202,NaN,NaN,Reimburse expenses,0,50
203,NaN,NaN,Uniforms - Reducing Balance,0,-33
204,NaN,NaN,Net Pay,0,13438.16
205,NaN,NaN,Superannuation,0,1363.42


In [ ]:
dic_emp_salary['002MORIC']

,0,1,5,10,11
207,NaN,NaN,Employee Previous Earnings,NaN,NaN
208,NaN,NaN,AMALGAMATED WIDGETS PTY LTD,NaN,NaN
209,Employee Previous Earnings,NaN,NaN,NaN,Page 5 of 553
210,Payroll Company,NaN,NaN,NaN,NaN
211,002MORIC,Crystal Morin,NaN,NaN,NaN
212,For Pay End Periods between 1/07/1982 and 25/0...,NaN,NaN,NaN,NaN
213,NaN,NaN,NaN,NaN,NaN
214,Period End Date,Pay Frequency,Description,Hours,Amount
215,1985/04/16,1W01 - Weekly,NaN,NaN,NaN
216,NaN,NaN,Normal Hours,46,1003.22


In [ ]:
dic_emp_salary['048LOWEP']

,0,1,5,10,11
30006,NaN,NaN,Employee Previous Earnings,NaN,NaN
30007,NaN,NaN,AMALGAMATED WIDGETS PTY LTD,NaN,NaN
30008,Employee Previous Earnings,NaN,NaN,NaN,Page 544 of 553
30009,Payroll Company,NaN,NaN,NaN,NaN
30010,048LOWEP,Patty Lowe,NaN,NaN,NaN
...,...,...,...,...,...
30545,NaN,NaN,SUPER SUPER VOL CONT $,0,-1.043751787246211
30546,NaN,NaN,Uniforms - Reducing Balance,0,-0.1789104947097512
30547,NaN,NaN,Net Pay,0,636.7198341435585
30548,NaN,NaN,Superannuation,0,68.33625107234788


In order to ease reshaping dataframe, it is better to save "employees total" and "companies total and average" data separatedly from employee salary description data.

In [ ]:
# creating dictionary with employee ids as keys and totals dataframe as values
dic_emp_total = {}

for emp_id, index in list_index_by_emp:
  df_emp = dic_emp_salary[emp_id]
  emp_total_index = df_emp[df_emp.loc[:,0] == 'Employee Total'].index[0]

  dic_emp_salary[emp_id] = df_emp.loc[:emp_total_index-1,:]
  dic_emp_total[emp_id] = df_emp.loc[emp_total_index:,:]

Let's see our results below for employee with the ID of "002MORIC".

In [ ]:
dic_emp_salary['002MORIC']

,0,1,5,10,11
207,NaN,NaN,Employee Previous Earnings,NaN,NaN
208,NaN,NaN,AMALGAMATED WIDGETS PTY LTD,NaN,NaN
209,Employee Previous Earnings,NaN,NaN,NaN,Page 5 of 553
210,Payroll Company,NaN,NaN,NaN,NaN
211,002MORIC,Crystal Morin,NaN,NaN,NaN
212,For Pay End Periods between 1/07/1982 and 25/0...,NaN,NaN,NaN,NaN
213,NaN,NaN,NaN,NaN,NaN
214,Period End Date,Pay Frequency,Description,Hours,Amount
215,1985/04/16,1W01 - Weekly,NaN,NaN,NaN
216,NaN,NaN,Normal Hours,46,1003.22


In [ ]:
dic_emp_total['002MORIC']

,0,1,5,10,11
239,Employee Total,NaN,NaN,NaN,NaN
240,NaN,NaN,Normal Hours,99.5,2170
241,NaN,NaN,Sat Casual Ldg,8.5,14.83
242,NaN,NaN,Gross Taxable Total,0,2184.83
243,NaN,NaN,Tax (Incl Adjust),0,-172
244,NaN,NaN,Net Pay,0,2012.8300000000002
245,NaN,NaN,Superannuation,0,199.26000000000002
246,ACME Payroll (Registered to AMALGAMATED CO. PT...,NaN,NaN,NaN,NaN


Now, let's split company total and company average data from the last employee's total data.

In [ ]:
# creating dictionary with "company_total" and "company_avg" as keys and respective values in dataframe as values
dic_company = {}

df_last_emp_total = dic_emp_total['048LOWEP']
company_total_index = df_last_emp_total[df_last_emp_total.loc[:,0] == 'Payroll Company Total'].index[0]
company_avg_index = df_last_emp_total[df_last_emp_total.loc[:,0] == 'Average'].index[0]

dic_company['company_total'] = df_last_emp_total.loc[company_total_index+1:company_avg_index-1,:]
dic_company['company_avg'] = df_last_emp_total.loc[company_avg_index+1:,:]
dic_emp_total['048LOWEP'] = df_last_emp_total.loc[:company_total_index-1,:]


Results of company total and company average below.

In [ ]:
dic_company['company_total']

,0,1,5,10,11
30443,NaN,NaN,Normal Hours,92486.75,2102742.6800000034
30444,NaN,NaN,Time/Half Hours,343.75,10633.759999999998
30445,NaN,NaN,Sick Leave,2520.3,61061.16000000009
30446,NaN,NaN,Annual Leave,6569.09,152592.36000000013
30447,NaN,NaN,Long Service Leave,639,14692.64
30448,NaN,NaN,Other Leave - Bereavement,48,1004.0799999999999
30449,NaN,NaN,Other Leave - Birthday,265,6260.930000000001
30450,NaN,NaN,Other Leave - Compassionate,9,367.79
30451,NaN,NaN,Other Leave - Jury Duty,104,2237.06
30452,NaN,NaN,Other Leave - Leave w/out pay- with accr,105.25,0


In [ ]:
dic_company['company_avg']

,0,1,5,10,11
30499,NaN,NaN,Normal Hours,26.447454961395483,601.2990219848743
30500,NaN,NaN,Time/Half Hours,0.0982985416070918,3.0408235630540457
30501,NaN,NaN,Sick Leave,0.7207034601086646,17.46101229625396
30502,NaN,NaN,Annual Leave,1.8784929939948527,43.63521875893627
30503,NaN,NaN,Long Service Leave,0.18272805261652844,4.201498427223334
30504,NaN,NaN,Other Leave - Bereavement,0.01372605090077209,0.2871261080926508
30505,NaN,NaN,Other Leave - Birthday,0.07577923934801258,1.7903717472118963
30506,NaN,NaN,Other Leave - Compassionate,0.002573634543894767,0.10517300543322848
30507,NaN,NaN,Other Leave - Jury Duty,0.02973977695167286,0.6397083214183585
30508,NaN,NaN,Other Leave - Leave w/out pay- with accr,0.03009722619388047,0


To make sure the last employee total details are correct, let's have a quick look.

In [ ]:
dic_emp_total['048LOWEP']

,0,1,5,10,11
30425,Employee Total,NaN,NaN,NaN,NaN
30426,NaN,NaN,Normal Hours,2032.5,69378.87999999998
30427,NaN,NaN,Sick Leave,24,819.24
30428,NaN,NaN,Annual Leave,199.5,6809.9
30429,NaN,NaN,Other Leave - Paternity Leave- 1 wk paid,40,1365.39
30430,NaN,NaN,Other Leave - Public Holiday,24,819.24
30431,NaN,NaN,Bonus,0,500
30432,NaN,NaN,Term AL Gross,173.71,5929.56
30433,NaN,NaN,Gross Taxable Total,0,85622.20999999998
30434,NaN,NaN,Tax (Incl Adjust),0,-20513


**THE 5TH AND 6TH STEPS APPLIED AS FOLLOWS:**
5. Removed unnecessary rows.
6. Removed unnecessary columns.

Let's keep only salary description value along with pay end dates, hours and amount in employee salary details dataframe.

**Rows to remove in employee salary details dataframe:** Rows with following text strings<br>
* Employee Previous Earnings
* Payroll Company
* For Pay End Periods between 1/07/1982 and 25/06/1985
* Period End Date
* Employee Previous Earnings
* AMALGAMATED WIDGETS PTY LTD

Also, we remove if all values in row are NaN.

Although we remove some rows, we should be aware of some values in it for the future. So that, let's keep it here.<br>

Periods interval: **For Pay End Periods between 1/07/1982 and 25/06/1985**


In [ ]:
# removing rows in df_emp_salary

for emp_id, index in list_index_by_emp:
  df_emp_salary = dic_emp_salary[emp_id]

  # remove rows included specific strings
  df_emp_salary.drop(df_emp_salary[df_emp_salary.iloc[:,0] == 'Employee Previous Earnings'].index, inplace = True)
  df_emp_salary.drop(df_emp_salary[df_emp_salary.iloc[:,0] == 'Payroll Company'].index, inplace = True)
  df_emp_salary.drop(df_emp_salary[df_emp_salary.iloc[:,0] == 'For Pay End Periods between 1/07/1982 and 25/06/1985'].index, inplace = True)
  df_emp_salary.drop(df_emp_salary[df_emp_salary.iloc[:,0] == 'Period End Date'].index, inplace = True)
  df_emp_salary.drop(df_emp_salary[df_emp_salary.iloc[:,0] == emp_id].index, inplace = True)

  df_emp_salary.drop(df_emp_salary[df_emp_salary.iloc[:,2] == 'Employee Previous Earnings'].index, inplace = True)
  df_emp_salary.drop(df_emp_salary[df_emp_salary.iloc[:,2] == 'AMALGAMATED WIDGETS PTY LTD'].index, inplace = True)

  # remove rows if all value NaN
  df_emp_salary.dropna(how='all', axis=0, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Let's check if unnecessary rows are removed in employee salary dataframe for the employee with ID of "002MORIC".

In [ ]:
dic_emp_salary['002MORIC']

,0,1,5,10,11
215,1985/04/16,1W01 - Weekly,NaN,NaN,NaN
216,NaN,NaN,Normal Hours,46,1003.22
217,NaN,NaN,Gross Taxable Total,0,1003.22
218,NaN,NaN,Tax (Incl Adjust),0,-85
219,NaN,NaN,Net Pay,0,918.22
220,NaN,NaN,Superannuation,0,95.30000000000001
221,1985/04/23,1W01 - Weekly,NaN,NaN,NaN
222,NaN,NaN,Normal Hours,19.5,425.27
223,NaN,NaN,Gross Taxable Total,0,425.27
224,NaN,NaN,Tax (Incl Adjust),0,-14


Let's keep only salary total description values along with hours and amount in employee totals dataframe.

**Rows to remove in employee total dataframe:** Rows with following text strings<br>
* Employee Total
* ACME Payroll (Registered to AMALGAMATED CO. PTY. LTD.)

In [ ]:
# removing rows in df_emp_total

for emp_id, index in list_index_by_emp:
  df_emp_total = dic_emp_total[emp_id]

  df_emp_total.drop(df_emp_total[df_emp_total.iloc[:,0] == 'Employee Total'].index, inplace = True)
  df_emp_total.drop(df_emp_total[df_emp_total.iloc[:,0] == 'ACME Payroll (Registered to AMALGAMATED CO. PTY. LTD.)'].index, inplace = True)

  # remove rows if all value NaN
  df_emp_total.dropna(how='all', axis=0, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Let's check if unnecessary rows are removed in employee total dataframe for the employee with ID of "002MORIC".

In [ ]:
dic_emp_total['002MORIC']

,0,1,5,10,11
240,NaN,NaN,Normal Hours,99.5,2170
241,NaN,NaN,Sat Casual Ldg,8.5,14.83
242,NaN,NaN,Gross Taxable Total,0,2184.83
243,NaN,NaN,Tax (Incl Adjust),0,-172
244,NaN,NaN,Net Pay,0,2012.8300000000002
245,NaN,NaN,Superannuation,0,199.26000000000002


Let's keep only company total and company average description values along with hours and amount in company total and company average dataframes.

**Rows to remove in company total and company average dataframes:** Rows with following text strings<br>
* ACME Payroll (Registered to AMALGAMATED CO. PTY. LTD.)
* Employee Previous Earnings
* Payroll Company
* AMALGAMATED WIDGETS PTY LTD


In [ ]:
# removing rows in df_company_total and df_company_avg

df_company_total = dic_company['company_total']
df_company_avg = dic_company['company_avg']

# remove rows included specific strings in df_company_total
df_company_total.drop(df_company_total[df_company_total.iloc[:,0] == 'ACME Payroll (Registered to AMALGAMATED CO. PTY. LTD.)'].index, inplace = True)
df_company_total.drop(df_company_total[df_company_total.iloc[:,0] == 'Employee Previous Earnings'].index, inplace = True)
df_company_total.drop(df_company_total[df_company_total.iloc[:,0] == 'Payroll Company'].index, inplace = True)

df_company_total.drop(df_company_total[df_company_total.iloc[:,2] == 'Employee Previous Earnings'].index, inplace = True)
df_company_total.drop(df_company_total[df_company_total.iloc[:,2] == 'AMALGAMATED WIDGETS PTY LTD'].index, inplace = True)

# remove rows if all value NaN in df_company_total
df_company_total.dropna(how='all', axis=0, inplace = True)


# remove rows included specific strings in df_company_avg
df_company_avg.drop(df_company_avg[df_company_avg.iloc[:,0] == 'ACME Payroll (Registered to AMALGAMATED CO. PTY. LTD.)'].index, inplace = True)
df_company_avg.drop(df_company_avg[df_company_avg.iloc[:,0] == 'Employee Previous Earnings'].index, inplace = True)
df_company_avg.drop(df_company_avg[df_company_avg.iloc[:,0] == 'Payroll Company'].index, inplace = True)

df_company_avg.drop(df_company_avg[df_company_avg.iloc[:,2] == 'Employee Previous Earnings'].index, inplace = True)
df_company_avg.drop(df_company_avg[df_company_avg.iloc[:,2] == 'AMALGAMATED WIDGETS PTY LTD'].index, inplace = True)

# remove rows if all value NaN in df_company_avg
df_company_avg.dropna(how='all', axis=0, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Let's check company total and company average dataframes to make sure all unnecessary rows are removed.

In [ ]:
dic_company['company_total']

,0,1,5,10,11
30443,NaN,NaN,Normal Hours,92486.75,2102742.6800000034
30444,NaN,NaN,Time/Half Hours,343.75,10633.759999999998
30445,NaN,NaN,Sick Leave,2520.3,61061.16000000009
30446,NaN,NaN,Annual Leave,6569.09,152592.36000000013
30447,NaN,NaN,Long Service Leave,639,14692.64
30448,NaN,NaN,Other Leave - Bereavement,48,1004.0799999999999
30449,NaN,NaN,Other Leave - Birthday,265,6260.930000000001
30450,NaN,NaN,Other Leave - Compassionate,9,367.79
30451,NaN,NaN,Other Leave - Jury Duty,104,2237.06
30452,NaN,NaN,Other Leave - Leave w/out pay- with accr,105.25,0


In [ ]:
dic_company['company_avg']

,0,1,5,10,11
30499,NaN,NaN,Normal Hours,26.447454961395483,601.2990219848743
30500,NaN,NaN,Time/Half Hours,0.0982985416070918,3.0408235630540457
30501,NaN,NaN,Sick Leave,0.7207034601086646,17.46101229625396
30502,NaN,NaN,Annual Leave,1.8784929939948527,43.63521875893627
30503,NaN,NaN,Long Service Leave,0.18272805261652844,4.201498427223334
30504,NaN,NaN,Other Leave - Bereavement,0.01372605090077209,0.2871261080926508
30505,NaN,NaN,Other Leave - Birthday,0.07577923934801258,1.7903717472118963
30506,NaN,NaN,Other Leave - Compassionate,0.002573634543894767,0.10517300543322848
30507,NaN,NaN,Other Leave - Jury Duty,0.02973977695167286,0.6397083214183585
30508,NaN,NaN,Other Leave - Leave w/out pay- with accr,0.03009722619388047,0


Let's remove unnecessary columns in all dataframes. <br>

*Employee salary details dataframe*<br>


**Columns to remove:** <br>
Column 1 - Payment frequency with only one unique value of **1W01 - Weekly**.

In [ ]:
for key in dic_emp_salary:
  dic_emp_salary[key] = dic_emp_salary[key].drop(dic_emp_salary[key].columns[[1]], axis=1)

dic_emp_salary['002MORIC']

,0,5,10,11
215,1985/04/16,NaN,NaN,NaN
216,NaN,Normal Hours,46,1003.22
217,NaN,Gross Taxable Total,0,1003.22
218,NaN,Tax (Incl Adjust),0,-85
219,NaN,Net Pay,0,918.22
220,NaN,Superannuation,0,95.30000000000001
221,1985/04/23,NaN,NaN,NaN
222,NaN,Normal Hours,19.5,425.27
223,NaN,Gross Taxable Total,0,425.27
224,NaN,Tax (Incl Adjust),0,-14



*Employee total dataframe*<br>


**Columns to remove:** <br>
Column 0 - Empty <br>
Column 1 - Empty

In [ ]:
for key in dic_emp_total:
  dic_emp_total[key] = dic_emp_total[key].drop(dic_emp_total[key].columns[[0,1]], axis=1)

dic_emp_total['002MORIC']


,5,10,11
240,Normal Hours,99.5,2170
241,Sat Casual Ldg,8.5,14.83
242,Gross Taxable Total,0,2184.83
243,Tax (Incl Adjust),0,-172
244,Net Pay,0,2012.8300000000002
245,Superannuation,0,199.26000000000002



*Company total and company average dataframes*<br>


**Columns to remove:** <br>
Column 0 - Empty <br>
Column 1 - Empty

In [ ]:
#remove columns by indexing
dic_company['company_total'] = dic_company['company_total'].drop(dic_company['company_total'].columns[[0,1]], axis=1)
dic_company['company_total']

,5,10,11
30443,Normal Hours,92486.75,2102742.6800000034
30444,Time/Half Hours,343.75,10633.759999999998
30445,Sick Leave,2520.3,61061.16000000009
30446,Annual Leave,6569.09,152592.36000000013
30447,Long Service Leave,639,14692.64
30448,Other Leave - Bereavement,48,1004.0799999999999
30449,Other Leave - Birthday,265,6260.930000000001
30450,Other Leave - Compassionate,9,367.79
30451,Other Leave - Jury Duty,104,2237.06
30452,Other Leave - Leave w/out pay- with accr,105.25,0


In [ ]:
dic_company['company_avg'] = dic_company['company_avg'].drop(dic_company['company_avg'].columns[[0,1]], axis=1)
dic_company['company_avg']

,5,10,11
30499,Normal Hours,26.447454961395483,601.2990219848743
30500,Time/Half Hours,0.0982985416070918,3.0408235630540457
30501,Sick Leave,0.7207034601086646,17.46101229625396
30502,Annual Leave,1.8784929939948527,43.63521875893627
30503,Long Service Leave,0.18272805261652844,4.201498427223334
30504,Other Leave - Bereavement,0.01372605090077209,0.2871261080926508
30505,Other Leave - Birthday,0.07577923934801258,1.7903717472118963
30506,Other Leave - Compassionate,0.002573634543894767,0.10517300543322848
30507,Other Leave - Jury Duty,0.02973977695167286,0.6397083214183585
30508,Other Leave - Leave w/out pay- with accr,0.03009722619388047,0


**THE 7TH STEP APPLIED AS FOLLOWS**
7. Filled empty dates in emp_salary_details dataframe using ffill() function to make next step (reshaping dataframe) easier.

In [ ]:
for key in dic_emp_salary:
  df_emp_salary = dic_emp_salary[key]
  dic_emp_salary[key][dic_emp_salary[key].columns[0]] = df_emp_salary[df_emp_salary.columns[0]].ffill()

In [ ]:
dic_emp_salary['001POWED']

,0,5,10,11
8,1982/12/06,NaN,NaN,NaN
9,1982/12/06,Normal Hours,23,351.87
10,1982/12/06,Gross Taxable Total,0,351.87
11,1982/12/06,Tax (Incl Adjust),0,-0
12,1982/12/06,Net Pay,0,351.87
...,...,...,...,...
191,1983/05/30,Normal Hours,25,446.21
192,1983/05/30,Gross Taxable Total,0,446.21
193,1983/05/30,Tax (Incl Adjust),0,-21
194,1983/05/30,Net Pay,0,425.21


In [ ]:
dic_emp_salary['002MORIC']

,0,5,10,11
215,1985/04/16,NaN,NaN,NaN
216,1985/04/16,Normal Hours,46,1003.22
217,1985/04/16,Gross Taxable Total,0,1003.22
218,1985/04/16,Tax (Incl Adjust),0,-85
219,1985/04/16,Net Pay,0,918.22
220,1985/04/16,Superannuation,0,95.30000000000001
221,1985/04/23,NaN,NaN,NaN
222,1985/04/23,Normal Hours,19.5,425.27
223,1985/04/23,Gross Taxable Total,0,425.27
224,1985/04/23,Tax (Incl Adjust),0,-14


**THE 8TH STEP APPLIED AS FOLLOWS**
8. Reshaped dataframes using unstack() function to make dataframe with a one header.

*Employee salary detail dataframe*

In [ ]:
col_names = ['pay_end_date', 'description', 'hours', 'amount']
for key in dic_emp_salary:
  # assign columns name
  dic_emp_salary[key].columns = col_names

  # TODO: check without dropna!
  dic_emp_salary[key] = dic_emp_salary[key].dropna()

  # set multi index
  dic_emp_salary[key] = dic_emp_salary[key].set_index(['pay_end_date', 'description'])

  # unstack dataframe - inner most level (in our case description)
  dic_emp_salary[key]  = dic_emp_salary[key].unstack()
  # flatten index into "hours_Net Pay", "amount_Net Pay" etc
  dic_emp_salary[key].columns = dic_emp_salary[key].columns.to_flat_index().str.join('_')

  # flatten the index of all levels
  dic_emp_salary[key].reset_index(inplace=True)

  # insert 2 new columns to save employee ids and names
  dic_emp_salary[key].insert(0, "id", key)
  dic_emp_salary[key].insert(1, "name", dic_emp_names[key])


In [ ]:
print("The 1st employee salary dataframe shape: ", dic_emp_salary['001POWED'].shape)
dic_emp_salary['001POWED']

The 1st employee salary dataframe shape:  (26, 21)


,id,name,pay_end_date,hours_Gross Taxable Total,hours_Net Pay,hours_Normal Hours,hours_Public Holiday - worked,hours_Reimburse expenses,hours_Superannuation,hours_Tax (Incl Adjust),...,hours_Uniforms - Reducing Balance,amount_Gross Taxable Total,amount_Net Pay,amount_Normal Hours,amount_Public Holiday - worked,amount_Reimburse expenses,amount_Superannuation,amount_Tax (Incl Adjust),amount_Time/Half Hours,amount_Uniforms - Reducing Balance
0,001POWED,Dominick Powers,1982/12/06,0,0,23,NaN,NaN,NaN,0,...,NaN,351.87,351.87,351.87,NaN,NaN,NaN,-0,NaN,NaN
1,001POWED,Dominick Powers,1982/12/13,0,0,46,NaN,NaN,0,0,...,NaN,703.74,623.74,703.74,NaN,NaN,66.86,-80,NaN,NaN
2,001POWED,Dominick Powers,1982/12/20,0,0,37.5,NaN,NaN,0,0,...,NaN,573.7,520.7,573.7,NaN,NaN,54.5,-53,NaN,NaN
3,001POWED,Dominick Powers,1982/12/27,0,0,30,15,NaN,0,0,...,NaN,1032.66,838.6600000000001,458.96,573.7,NaN,98.1,-194,NaN,NaN
4,001POWED,Dominick Powers,1983/01/03,0,0,37.5,NaN,NaN,0,0,...,NaN,573.7,520.7,573.7,NaN,NaN,54.5,-53,NaN,NaN
5,001POWED,Dominick Powers,1983/01/10,0,0,32,NaN,NaN,0,0,...,NaN,615.7700000000001,553.7700000000001,489.56000000000006,NaN,NaN,46.51,-62,126.21,NaN
6,001POWED,Dominick Powers,1983/01/17,0,0,31,NaN,NaN,0,0,...,NaN,474.26,445.26,474.26,NaN,NaN,45.05,-29,NaN,NaN
7,001POWED,Dominick Powers,1983/01/24,0,0,37,NaN,NaN,0,0,...,NaN,566.05,514.05,566.05,NaN,NaN,53.77,-52,NaN,NaN
8,001POWED,Dominick Powers,1983/01/31,0,0,33.5,NaN,NaN,0,0,...,NaN,512.51,472.51,512.51,NaN,NaN,48.69,-40,NaN,NaN
9,001POWED,Dominick Powers,1983/02/07,0,0,35.5,NaN,0,0,0,...,NaN,543.1,546.1,543.1,NaN,50,51.59,-47,NaN,NaN


In [ ]:
print("The 2nd employee salary dataframe shape: ", dic_emp_salary['002MORIC'].shape)
dic_emp_salary['002MORIC']

The 2nd employee salary dataframe shape:  (4, 15)


,id,name,pay_end_date,hours_Gross Taxable Total,hours_Net Pay,hours_Normal Hours,hours_Sat Casual Ldg,hours_Superannuation,hours_Tax (Incl Adjust),amount_Gross Taxable Total,amount_Net Pay,amount_Normal Hours,amount_Sat Casual Ldg,amount_Superannuation,amount_Tax (Incl Adjust)
0,002MORIC,Crystal Morin,1985/04/16,0,0,46,NaN,0,0,1003.22,918.22,1003.22,NaN,95.30000000000001,-85
1,002MORIC,Crystal Morin,1985/04/23,0,0,19.5,NaN,0,0,425.27,411.27,425.27,NaN,40.4,-14
2,002MORIC,Crystal Morin,1985/04/30,0,0,30,8.5,0,0,669.1,596.1,654.27,14.83,63.56,-73
3,002MORIC,Crystal Morin,1985/05/07,0,0,4,NaN,NaN,0,87.24,87.24,87.24,NaN,NaN,-0


*Employee total dataframe*

In [ ]:
col_names = ['description', 'hours', 'amount']
for key in dic_emp_total:
  dic_emp_total[key].columns = col_names
  dic_emp_total[key] = dic_emp_total[key].dropna()
  dic_emp_total[key] = dic_emp_total[key].set_index(['description'])

  dic_emp_total[key]  = dic_emp_total[key].unstack().to_frame().transpose()

  dic_emp_total[key].columns = dic_emp_total[key].columns.to_flat_index().str.join('_')

In [ ]:
print("The 1st employee total dataframe shape: ", dic_emp_total['001POWED'].shape)
dic_emp_total['001POWED']

The 1st employee total dataframe shape:  (1, 18)


,hours_Normal Hours,hours_Time/Half Hours,hours_Public Holiday - worked,hours_Gross Taxable Total,hours_Tax (Incl Adjust),hours_Reimburse expenses,hours_Uniforms - Reducing Balance,hours_Net Pay,hours_Superannuation,amount_Normal Hours,amount_Time/Half Hours,amount_Public Holiday - worked,amount_Gross Taxable Total,amount_Tax (Incl Adjust),amount_Reimburse expenses,amount_Uniforms - Reducing Balance,amount_Net Pay,amount_Superannuation
0,852,5.5,23.5,0,0,0,0,0,0,13750.97,126.21,952.98,14830.16,-1409,50,-33,13438.16,1363.42


In [ ]:
print("The last employee total dataframe shape: ", dic_emp_total['048LOWEP'].shape)
dic_emp_total['048LOWEP']

The last employee total dataframe shape:  (1, 22)


,hours_Normal Hours,hours_Sick Leave,hours_Annual Leave,hours_Other Leave - Paternity Leave- 1 wk paid,hours_Other Leave - Public Holiday,hours_Bonus,hours_Term AL Gross,hours_Gross Taxable Total,hours_Tax (Incl Adjust),hours_Net Pay,...,amount_Sick Leave,amount_Annual Leave,amount_Other Leave - Paternity Leave- 1 wk paid,amount_Other Leave - Public Holiday,amount_Bonus,amount_Term AL Gross,amount_Gross Taxable Total,amount_Tax (Incl Adjust),amount_Net Pay,amount_Superannuation
0,2032.5,24,199.5,40,24,0,173.71,0,0,0,...,819.24,6809.9,1365.39,819.24,500,5929.56,85622.20999999998,-20513,65109.20999999998,7393.470000000001


*Company total and company average dataframes*

In [ ]:
col_names = ['description', 'hours', 'amount']
for key in dic_company:
  dic_company[key].columns = col_names
  dic_company[key] = dic_company[key].dropna()
  dic_company[key] = dic_company[key].set_index(['description'])

  dic_company[key]  = dic_company[key].unstack().to_frame().transpose()

  dic_company[key].columns = dic_company[key].columns.to_flat_index().str.join('_')

In [ ]:
print("The company total dataframe shape: ", dic_company['company_total'].shape)
dic_company['company_total']

The company total dataframe shape:  (1, 96)


,hours_Normal Hours,hours_Time/Half Hours,hours_Sick Leave,hours_Annual Leave,hours_Long Service Leave,hours_Other Leave - Bereavement,hours_Other Leave - Birthday,hours_Other Leave - Compassionate,hours_Other Leave - Jury Duty,hours_Other Leave - Leave w/out pay- with accr,...,amount_Re-imburse expense - Safety Boots,amount_Reimburse expenses,amount_Reimburse expenses- Flu Vax,amount_Account Payment,amount_Jury Duty Subsidy,amount_Loan Repayments - Reducing balance,amount_SUPER SUPER VOL CONT $,amount_Uniforms - Reducing Balance,amount_Net Pay,amount_Superannuation
0,92486.75,343.75,2520.3,6569.09,639,48,265,9,104,105.25,...,200,1034.98,92.92,-4175.55,-300,-350,-3650,-625.65,2226609.260000024,238971.87000000052


In [ ]:
print("The company average dataframe shape: ", dic_company['company_avg'].shape)
dic_company['company_avg']

The company average dataframe shape:  (1, 96)


,hours_Normal Hours,hours_Time/Half Hours,hours_Sick Leave,hours_Annual Leave,hours_Long Service Leave,hours_Other Leave - Bereavement,hours_Other Leave - Birthday,hours_Other Leave - Compassionate,hours_Other Leave - Jury Duty,hours_Other Leave - Leave w/out pay- with accr,...,amount_Re-imburse expense - Safety Boots,amount_Reimburse expenses,amount_Reimburse expenses- Flu Vax,amount_Account Payment,amount_Jury Duty Subsidy,amount_Loan Repayments - Reducing balance,amount_SUPER SUPER VOL CONT $,amount_Uniforms - Reducing Balance,amount_Net Pay,amount_Superannuation
0,26.447454961395483,0.0982985416070918,0.7207034601086646,1.8784929939948527,0.18272805261652844,0.01372605090077209,0.07577923934801258,0.002573634543894767,0.02973977695167286,0.03009722619388047,...,0.057191878753217046,0.2959622533600229,0.026571346868744638,-1.1940377466399772,-0.08578781812982557,-0.10008578781812982,-1.043751787246211,-0.1789104947097512,636.7198341435585,68.33625107234788


**THE 9TH AND 10TH STEPS APPLIED AS FOLLOWS**
9. Concatenated all dataframes to the one big dataframe.
10. Exported resulted dataframe into CSV.

In [ ]:
list_df_all_emp = []
for key in dic_emp_salary:
  list_df_all_emp.append(dic_emp_salary[key])
  list_df_all_emp.append(dic_emp_total[key])

list_df_all_emp.append(dic_company['company_total'])
list_df_all_emp.append(dic_company['company_avg'])

result = pd.concat(list_df_all_emp, ignore_index=True, sort=False)

In [ ]:
print("The result dataframe shape: ", result.shape)
result.head(32)

The result dataframe shape:  (3509, 103)


,id,name,pay_end_date,hours_Gross Taxable Total,hours_Net Pay,hours_Normal Hours,hours_Public Holiday - worked,hours_Reimburse expenses,hours_Superannuation,hours_Tax (Incl Adjust),...,hours_Salary Sacrifice - Super Superannuation,amount_Bonus,amount_Other Leave - Workcover- lve accrued,amount_Salary Sacrifice - Super Superannuation,hours_Salary Sacrifice - Carcosan Super,amount_Salary Sacrifice - Carcosan Super,hours_Other Leave - Compassionate,amount_Other Leave - Compassionate,hours_Other Leave - Paternity Leave- 1 wk paid,amount_Other Leave - Paternity Leave- 1 wk paid
0,001POWED,Dominick Powers,1982/12/06,0,0,23,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001POWED,Dominick Powers,1982/12/13,0,0,46,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001POWED,Dominick Powers,1982/12/20,0,0,37.5,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001POWED,Dominick Powers,1982/12/27,0,0,30,15,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001POWED,Dominick Powers,1983/01/03,0,0,37.5,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,001POWED,Dominick Powers,1983/01/10,0,0,32,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,001POWED,Dominick Powers,1983/01/17,0,0,31,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,001POWED,Dominick Powers,1983/01/24,0,0,37,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,001POWED,Dominick Powers,1983/01/31,0,0,33.5,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,001POWED,Dominick Powers,1983/02/07,0,0,35.5,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# export result dataframe to CSV file
result.to_csv('/content/result.csv')

### FURTHER IMPROVEMENTS


*   Find out all missing values
*   Find out all duplicated values and remove duplications
*   Remove column names auto sorting
*   Rename column names
*   Remove columns if values are all 0, NaN or None
*   Make easier visualisation where client can filter data by name, date or certain column names.
*   Provide report by each employee if it is necessary.
*   Code review to make it efficient.





Here is the end of my task. Thank you for your time go through all the work and I hope the team see me to discuss about the approach.